# 并行计算
- 数据在GPU和CPU之间进行传递时会比较耗时，我们应当尽量避免数据的切换
- GPU运算很快，但是在使用简单的操作时，应该尽量使用CPU去完成
- 服务器上有多个GPU，应该指明我们使用的GPU是哪一块，如果不设置，tensor.cuda()方法会默认将tensor保存到第一块GPU上，等价于tensor.cuda(0)，这将有可能导致爆出out of memory的错误

两种设置方法

In [ ]:
#设置在文件最开始部分
import os
os.environ["CUDA_VISIBLE_DEVICE"] = "2" # 设置默认的显卡

In [ ]:
!CUDA_VISBLE_DEVICE=0,1 python train.py # 使用0，1两块GPU

## 使用CUDA加速训练

### 1. 单卡训练

In [ ]:
model = Net()
model.cuda() # 模型显示转移到CUDA上

for image,label in dataloader:
    # 图像和标签显示转移到CUDA上
    image = image.cuda() 
    label = label.cuda()

### 2. 多卡训练

PyTorch提供了两种多卡训练的方式，分别为DataParallel和DistributedDataParallel（以下我们分别简称为DP和DDP）。这两种方法中官方更推荐我们使用DDP，因为它的性能更好。但是DDP的使用比较复杂，而DP经需要改变几行代码既可以实现

#### 单机多卡DP
通常使用一种叫做数据并行 (Data parallelism) 的策略，即将计算任务划分成多个子任务并在多个GPU卡上同时执行这些子任务

In [ ]:
model = Net()
model.cuda() # 模型显示转移到CUDA上

if torch.cuda.device_count() > 1: # 含有多张GPU的卡
	model = nn.DataParallel(model) # 单机多卡DP训练

nn.DataParallel函数传入device_ids参数，可以指定了使用的GPU编号

In [ ]:
model = nn.DataParallel(model, device_ids=[0,1]) # 使用第0和第1张卡进行并行训练

要手动指定对程序可见的GPU设备

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

#### 多机多卡DDP
通过DP进行分布式多卡训练的方式容易造成负载不均衡，有可能第一块GPU显存占用更多，因为输出默认都会被gather到第一块GPU上。为此Pytorch也提供了torch.nn.parallel.DistributedDataParallel（DDP）方法来解决这个问题。

针对每个GPU，启动一个进程，然后这些进程在最开始的时候会保持一致（模型的初始化参数也一致，每个进程拥有自己的优化器），同时在更新模型的时候，梯度传播也是完全一致的，这样就可以保证任何一个GPU上面的模型参数就是完全一致的，所以这样就不会出现DataParallel那样显存不均衡的问题。不过相对应的，会比较麻烦，接下来介绍一下多机多卡DDP的使用方法。

进程组的相关概念

GROUP：进程组，默认情况下，只有一个组，一个 job 即为一个组，也即一个 world。（当需要进行更加精细的通信时，可以通过 new_group 接口，使用 world 的子集，创建新组，用于集体通信等。）

WORLD_SIZE：表示全局进程个数。如果是多机多卡就表示机器数量，如果是单机多卡就表示 GPU 数量。

RANK：表示进程序号，用于进程间通讯，表征进程优先级。rank = 0 的主机为 master 节点。 如果是多机多卡就表示对应第几台机器，如果是单机多卡，由于一个进程内就只有一个 GPU，所以 rank 也就表示第几块 GPU。

LOCAL_RANK：表示进程内，GPU 编号，非显式参数，由 torch.distributed.launch 内部指定。例如，多机多卡中 rank = 3，local_rank = 0 表示第 3 个进程内的第 1 块 GPU。

DDP的基本用法 (代码编写流程)

在使用 distributed 包的任何其他函数之前，需要使用 init_process_group 初始化进程组，同时初始化 distributed 包。

使用 torch.nn.parallel.DistributedDataParallel 创建 分布式模型 DDP(model, device_ids=device_ids)

使用 torch.utils.data.distributed.DistributedSampler 创建 DataLoader

使用启动工具 torch.distributed.launch 在每个主机上执行一次脚本，开始训练

首先是对代码进行修改，添加参数 --local_rank

In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--local_rank", type=int) # 这个参数很重要
args = parser.parse_args()

这里的local_rank参数，可以理解为torch.distributed.launch在给一个GPU创建进程的时候，给这个进程提供的GPU号，这个是程序自动给的，不需要手动在命令行中指定这个参数

In [ ]:
local_rank = int(os.environ["LOCAL_RANK"]) #也可以自动获取

然后在所有和GPU相关代码的前面添加如下代码，如果不写这句代码，所有的进程都默认在你使用CUDA_VISIBLE_DEVICES参数设定的0号GPU上面启动

In [ ]:
torch.cuda.set_device(args.local_rank) # 调整计算的位置

初始化backend，也就是俗称的后端  

经验之谈

如果是使用cpu的分布式计算, 建议使用gloo，因为表中可以看到 gloo对cpu的支持是最好的

如果使用gpu进行分布式计算, 建议使用nccl。

GPU主机

InfiniBand连接，建议使用nccl，因为它是目前唯一支持 InfiniBand 和 GPUDirect 的后端。

Ethernet连接，建议使用nccl，因为它的分布式GPU训练性能目前是最好的，特别是对于多进程单节点或多节点分布式训练。 如果在使用 nccl时遇到任何问题，可以使用gloo 作为后备选项。 （不过注意，对于 GPU，gloo 目前的运行速度比 nccl 慢。）

CPU主机

InfiniBand连接，如果启用了IP over IB，那就使用gloo，否则使用mpi

Ethernet连接，建议使用gloo，除非有不得已的理由使用mpi。

当后端选择好了之后, 我们需要设置一下网络接口, 因为多个主机之间肯定是使用网络进行交换, 那肯定就涉及到IP之类的, 对于nccl和gloo一般会自己寻找网络接口，不过有时候如果网卡比较多的时候，就需要自己设置，可以利用以下代码

In [ ]:
import os
# 以下二选一, 第一个是使用gloo后端需要设置的, 第二个是使用nccl需要设置的
os.environ['GLOO_SOCKET_IFNAME'] = 'eth0'
os.environ['NCCL_SOCKET_IFNAME'] = 'eth0'

可以通过以下操作知道自己的网络接口，输入ifconfig, 然后找到自己IP地址的就是, 一般就是em0, eth0, esp2s0之类的,



从以上介绍我们可以看出， 当使用GPU的时候, nccl的效率是高于gloo的，我们一般还是会选择nccl后端，设置GPU之间通信使用的后端和端口：

In [ ]:
# ps 检查nccl是否可用
# torch.distributed.is_nccl_available ()
torch.distributed.init_process_group(backend='nccl') # 选择nccl后端，初始化进程组

之后，使用 DistributedSampler 对数据集进行划分。它能帮助我们将每个 batch 划分成几个 partition，在当前进程中只需要获取和 rank 对应的那个 partition 进行训练

In [ ]:
# 创建Dataloader
train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, sampler=train_sampler)

注意： testset不用sampler

然后使用torch.nn.parallel.DistributedDataParallel包装模型：

In [ ]:
# DDP进行训练
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank])

启动DDP，这不同于DP的方式，需要使用torch.distributed.launch启动器，对于单机多卡的情况

In [ ]:
CUDA_VISIBLE_DEVICES=0,1,2,3 python -m torch.distributed.launch --nproc_per_node=4 main.py
# nproc_per_node: 这个参数是指你使用这台服务器上面的几张显卡